In [2]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE, BorderlineSMOTE
from imblearn.under_sampling import RandomUnderSampler, TomekLinks
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, classification_report, make_scorer, accuracy_score
from sklearn.model_selection import RandomizedSearchCV, KFold, train_test_split
from collections import Counter
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

import torch
import torchvision
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import time

%matplotlib inline

# Load Data:

In [285]:
X_train = pd.read_csv('Data/X_train_mlp.csv')
y_train = pd.read_csv('Data/y_train_mlp.csv', header = None)

# Splitting Out 10% to Hyperparameter Search:

In [286]:
# Converting y_train into an array:
y_train = np.array(y_train[0])

In [287]:
# Using train-test split to split out a random, stratified 10% portion of the training data to grid search:
X_gs, X_leave, y_gs, y_leave = train_test_split(X_train, y_train, 
                                                test_size = 0.9,     # Inverse of the size of the data to search
                                                stratify = y_train,  # Keeping same proportion of target classes
                                                random_state = 42)  

In [288]:
# Converting X to np array:
X_gs = X_gs.to_numpy()

# Distribution of Forest Cover Types in Dataset:

In [289]:
y_gs

array([1, 2, 2, ..., 1, 1, 2], dtype=int64)

In [290]:
y_gs.tolist().count(1)

13558

In [291]:
unique, counts = np.unique(y_gs, return_counts=True)

print(np.asarray((unique, counts)).T)

[[    1 13558]
 [    2 18131]
 [    3  2288]
 [    4   176]
 [    5   607]
 [    6  1111]
 [    7  1313]]


# Other Functions:

In [292]:
class CreatePytorchDataset(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
    
    def __len__(self):
        return len(self.X_data)
    
    def __getitem__(self, idx):
        return self.X_data[idx], self.y_data[idx]

In [293]:
def calc_acc(y_pred, y_test):
    y_pred_torch = torch.log_softmax(y_pred, dim = 1)
    _, y_pred_values = torch.max(y_pred_torch, dim = 1)    
    correct_pred = (y_pred_values == y_test).float()
    accuracy = correct_pred.sum() / len(correct_pred)
    accuracy = accuracy * 100
    
    return accuracy

# MLP Model:

In [294]:
class MLP(nn.Module):
    
    def __init__(self, n_features, n_classes, n_neurons, activ, drop):
        super(MLP, self).__init__()
        
        # Define the layers: 
        # Defining the linear tranformations:
        self.fc1 = nn.Linear(n_features, n_neurons)
        self.fc2 = nn.Linear(n_neurons, int(n_neurons/2))
        self.fc3 = nn.Linear(int(n_neurons/2), int(n_neurons/4))
        
        # Output layer
        self.output = nn.Linear(int(n_neurons/4), n_classes)
       
        # Batch normalisation layers 
        # (https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html?highlight=batchnorm#torch.nn.BatchNorm1d)
        self.batchnorm1 = nn.BatchNorm1d(n_neurons)
        self.batchnorm2 = nn.BatchNorm1d(int(n_neurons/2))
        self.batchnorm3 = nn.BatchNorm1d(int(n_neurons/4))
    
        # Defining dropout rate:
        self.dropout = nn.Dropout(drop) 
        
        # Defining activation function
        self.activ = activ
        
        
    # Define forward function
    def forward(self, x):
        hidden = self.fc1(x)
        hidden = self.batchnorm1(hidden)
        hidden = self.activ(hidden)
        
        hidden = self.fc2(hidden)
        hidden = self.batchnorm2(hidden)
        hidden = self.activ(hidden)  
        hidden = self.dropout(hidden)
        
        hidden = self.fc3(hidden)
        hidden = self.batchnorm3(hidden)
        hidden = self.activ(hidden)
        hidden = self.dropout(hidden)
        
        output = self.output(hidden)
        
        return output
    

# Cross Validation Loop:

In [303]:
def cross_val_loop(model, n_epochs, train_loader, val_loader, criterion, optimizer):
    
    # To track lowest validation loss through epochs, set initially to infinity:
    min_val_loss = np.Inf
    max_acc = 0
    
    # List to hold losses through epochs:
    val_loss_list = []
    
    for epoch in range(1, n_epochs+1):
        
        # Setting initial losses/accuracy to zero, to be updated within batch loops
        val_loss_epoch = 0
        val_acc_epoch = 0

        # TRAINING #
        model.train()

        for data, target in train_loader:
            # Set gradients of model parameters to zero:
            optimizer.zero_grad()

            # Making predictions based on the training data batch
            y_pred_batch = model(data)

            # Cross-Entropy Loss:
            train_batch_loss = criterion(y_pred_batch, target.flatten())

            # Classifications correct within batch, to use to calculate overall accuracy:
            train_batch_acc = calc_acc(y_pred_batch, target.flatten())

            # Calculate the sum of gradients using backward:
            train_batch_loss.backward()

            # Update parameters by performing a single optimization:
            optimizer.step()


        # Disabling gradient calculation to speed up process, as backward will not be called during validation:
        with torch.no_grad():
            # VALIDATION #
            model.eval()

            for data, target in val_loader:
                # Use the model to predict target in val set batch:
                y_pred_val_batch =  model(data)

                # Calculate loss and accuracy on val set:
                val_batch_loss = criterion(y_pred_val_batch, target.flatten())
                val_batch_acc = calc_acc(y_pred_val_batch, target.flatten())

                # Update the running loss/accuracy totals:
                val_loss_epoch += train_batch_loss.item()
                val_acc_epoch += train_batch_acc.item()

        # PRINTING STATISTICS #

        # Calculate metrics across epoch:

        avg_val_loss = val_loss_epoch/ len(val_loader)
        avg_val_acc = val_acc_epoch/ len(val_loader)

        # Appending lists outside epoch loop to store results:
        val_loss_list.append(avg_val_loss)
        
        if avg_val_loss <= min_val_loss:
            # Alter min loss to current avg_val_loss:
            min_val_loss = avg_val_loss
            
        if avg_val_acc >= max_acc:
            # Alter min loss to current avg_val_loss:
            max_acc = avg_val_acc
        
        # EARLY STOPPING CRITERIA #
        # If validation loss has increased for 5 consequetive epochs, end loop:
        # Only run if > 10 epochs gone
        if epoch > 10:
            # Checks if the last 5 val_losses are all larger than the preceeding one:
            if ((val_loss_list[-1]>val_loss_list[-2]) and (val_loss_list[-2]>val_loss_list[-3]) and 
                (val_loss_list[-3]>val_loss_list[-4]) and (val_loss_list[-4]>val_loss_list[-5]) and 
                (val_loss_list[-5]>val_loss_list[-6])):
                break 
            

    return min_val_loss, max_acc, epoch

# Parameter Search Using Over and Under Sampling Methods:

In [298]:
# Setting static variables:

N_CLASSES = 7
N_INPUT_FEAT = 54
CRITERION = nn.CrossEntropyLoss()
EVAL_METRIC = make_scorer(accuracy_score)
max_epochs = 100
n_folds = 4
kf = KFold(n_splits = n_folds,shuffle = False)

In [299]:
# Setting Model and Optimizer (parameters found previously):

mlp_model = MLP(n_features=54, n_classes=7, n_neurons=512, activ = nn.LeakyReLU(), drop = 0.1)

optimizer = torch.optim.SGD(mlp_model.parameters(), lr = 0.1, momentum = 0.9)

## Defining Sampling Strategies:

In [300]:
# Sampling strategies to loop through:
tomek = [TomekLinks(sampling_strategy=[1,2]), 'None']

# Undersampling the two majority classes - Types 1 & 2:
unders = [(0.85, 0.85),                   # 15 % undersampling of majority classes
          (0.75, 0.75),                   # 25 %
          (0.60, 0.60),                   # 40 %
          (0.40, 0.40),                   # 60 %
          (0.20, 0.20),                   # 80 %
          (0.55, 0.40)  ]                 # Imbalanced undersampling

# Oversampling the 5 minority classes - Types 3, 4, 5, 6 & 7:
overs = [(1.2, 1.2, 1.2, 1.2, 1.2),       # 20% oversampling
         (1.5, 1.5, 1.5, 1.5, 1.5),       # 50% oversampling
         (2.0, 2.0, 2.0, 2.0, 2.0),       # 100% oversampling
         (4.0, 4.0, 4.0, 4.0, 4.0),       # 300% oversampling
         (6.0, 6.0, 6.0, 6.0, 6.0),       # 500% oversampling
         (2.0, 10.0, 10.0, 5.0, 4.0),     # Imbalanced oversampling
         (0),
         (1.2, 1.2, 1.2, 1.2, 1.2),       # Borderline SMOTE
         (1.5, 1.5, 1.5, 1.5, 1.5),
         (2.0, 2.0, 2.0, 2.0, 2.0),
         (4.0, 4.0, 4.0, 4.0, 4.0),
         (6.0, 6.0, 6.0, 6.0, 6.0),
         (2.0, 10.0, 10.0, 5.0, 4.0),
         (0),  ]

In [301]:
def get_count(array, cov_type, percent):
        # Count the number of a specific cover type in the current split array to define sampling bounds:
        count = round(array.tolist().count(cov_type)*percent)
        return count

In [315]:
test_results = np.zeros(((len(tomek)*len(unders)*len(overs)), 11))

loop_counter = 0

print("TOTAL LOOPS = ", (len(tomek)*len(unders)*len(overs)*n_folds))


for t in range(len(tomek)):

    for u in range(len(unders)):

        for o in range(len(overs)):

            val_losses = 0
            accuracies = 0
            epochs = 0
            vll = []

            start = time.time()

            for train_index, test_index in kf.split(X_gs):
                
                # Setting Model and Optimizer (parameters found previously):
                mlp_model = MLP(n_features=54, n_classes=7, n_neurons=512, activ = nn.LeakyReLU(), drop = 0.1)
                optimizer = torch.optim.SGD(mlp_model.parameters(), lr = 0.1, momentum = 0.9)

                # Obtaining the indexes to split the data for each Cross Validation Fold:
                X_train_CV, X_val_CV = X_gs[train_index], X_gs[test_index]
                y_train_CV, y_val_CV = y_gs[train_index], y_gs[test_index]
                
                # Definining Scaler:
                scaler = MinMaxScaler()

                # Fitting scaler on X_train_CV:
                X_train_CV = scaler.fit_transform(X_train_CV)
                
                # The applying that scaler to X_val separately:
                X_val_CV = scaler.transform(X_val_CV)
                
                
                ###### APPLYING SAMPLING STRATEGY ######
                # Appling sampling strategies to X_train/y_train after splitting for Cross-val
                
                # Only applying Tomek Links on half the loops (t = 0 or 1):
                if t == 0:
                    resample = tomek[t]
                    X_train_CV, y_train_CV = resample.fit_resample(X_train_CV, y_train_CV)
                    
                # Undersampling applied on majority classes according to methods above:
                under_strategy = {1:get_count(y_train_CV, 1, unders[u][0]),
                                  2:get_count(y_train_CV, 2, unders[u][1])}
                
                undersample = RandomUnderSampler(sampling_strategy = under_strategy)
                X_train_CV, y_train_CV = undersample.fit_resample(X_train_CV, y_train_CV)
                
                # Oversampling applied on minority classes according to methods above:
                if o == 6:
                    oversample = SMOTE()
                elif o == 13:
                    oversample = BorderlineSMOTE()
                elif o < 6:
                    over_strategy = {3:get_count(y_train_CV, 3, overs[o][0]),
                                     4:get_count(y_train_CV, 4, overs[o][1]),
                                     5:get_count(y_train_CV, 5, overs[o][2]),
                                     6:get_count(y_train_CV, 6, overs[o][3]),
                                     7:get_count(y_train_CV, 7, overs[o][4]),
                                    }
                    oversample = SMOTE(sampling_strategy = over_strategy)
                else:
                    over_strategy = {3:get_count(y_train_CV, 3, overs[o][0]),
                                     4:get_count(y_train_CV, 4, overs[o][1]),
                                     5:get_count(y_train_CV, 5, overs[o][2]),
                                     6:get_count(y_train_CV, 6, overs[o][3]),
                                     7:get_count(y_train_CV, 7, overs[o][4]),
                                    }
                    oversample = BorderlineSMOTE(sampling_strategy = over_strategy)
                
                X_train_CV, y_train_CV = oversample.fit_resample(X_train_CV, y_train_CV)
        

                # Converting these fold splits into tensors:
                X_train_CV_tensor = torch.from_numpy(X_train_CV).float()
                X_val_CV_tensor = torch.from_numpy(X_val_CV).float()
                y_train_CV_tensor = torch.from_numpy(y_train_CV).long() - 1
                y_val_CV_tensor = torch.from_numpy(y_val_CV).long() - 1

                # Creating combined datasets to feed into Data Loader:
                train_CV_dataset = CreatePytorchDataset(X_train_CV_tensor, y_train_CV_tensor)
                val_CV_dataset = CreatePytorchDataset(X_val_CV_tensor, y_val_CV_tensor)

                # Creating Data Loader Objects:
                train_CV_loader = DataLoader(train_CV_dataset, batch_size=512, shuffle = True, drop_last=True)
                val_CV_loader = DataLoader(val_CV_dataset, batch_size=512, shuffle=True, drop_last=True)


                print("Starting Loop {:.0f}".format(loop_counter))
                loop_counter += 1

                ######## EPOCH MODEL TRAINING ########
                min_val_loss_epochs, max_acc_epochs, epochs_taken = cross_val_loop(mlp_model, max_epochs, 
                                                     train_CV_loader, val_CV_loader, CRITERION, optimizer)

                val_losses += min_val_loss_epochs
                accuracies += max_acc_epochs
                epochs += epochs_taken
                vll.append(min_val_loss_epochs)

                #print("Time to complete CV Loop: {:.1f} minutes".format((time.process_time() - start)/60))

            av_val_loss = val_losses/ n_folds
            av_accuracy = accuracies/ n_folds
            av_epochs = epochs / n_folds
            time_taken = time.time() - start

            test_results[o+(u*len(overs))+(t*(len(unders)*len(overs)))][0] = av_val_loss
            test_results[o+(u*len(overs))+(t*(len(unders)*len(overs)))][1] = av_accuracy
            test_results[o+(u*len(overs))+(t*(len(unders)*len(overs)))][2] = t
            test_results[o+(u*len(overs))+(t*(len(unders)*len(overs)))][3] = u
            test_results[o+(u*len(overs))+(t*(len(unders)*len(overs)))][4] = o
            test_results[o+(u*len(overs))+(t*(len(unders)*len(overs)))][5] = time_taken
            test_results[o+(u*len(overs))+(t*(len(unders)*len(overs)))][6] = av_epochs
            test_results[o+(u*len(overs))+(t*(len(unders)*len(overs)))][7] = vll[0]
            test_results[o+(u*len(overs))+(t*(len(unders)*len(overs)))][8] = vll[1]
            test_results[o+(u*len(overs))+(t*(len(unders)*len(overs)))][9] = vll[2]
            test_results[o+(u*len(overs))+(t*(len(unders)*len(overs)))][10] = vll[3]


TOTAL LOOPS =  672
Starting Loop 0
Starting Loop 1
Starting Loop 2
Starting Loop 3
Starting Loop 4
Starting Loop 5
Starting Loop 6
Starting Loop 7
Starting Loop 8
Starting Loop 9
Starting Loop 10
Starting Loop 11
Starting Loop 12
Starting Loop 13
Starting Loop 14
Starting Loop 15
Starting Loop 16
Starting Loop 17
Starting Loop 18
Starting Loop 19
Starting Loop 20
Starting Loop 21
Starting Loop 22
Starting Loop 23
Starting Loop 24
Starting Loop 25
Starting Loop 26
Starting Loop 27
Starting Loop 28
Starting Loop 29
Starting Loop 30
Starting Loop 31
Starting Loop 32
Starting Loop 33
Starting Loop 34
Starting Loop 35
Starting Loop 36
Starting Loop 37
Starting Loop 38
Starting Loop 39
Starting Loop 40
Starting Loop 41
Starting Loop 42
Starting Loop 43
Starting Loop 44
Starting Loop 45
Starting Loop 46
Starting Loop 47
Starting Loop 48
Starting Loop 49
Starting Loop 50
Starting Loop 51
Starting Loop 52
Starting Loop 53
Starting Loop 54
Starting Loop 55
Starting Loop 56
Starting Loop 57
Start

C:\Users\rozhouse\Anaconda3\lib\site-packages\imblearn\utils\_validation.py:326: UserWarning: After over-sampling, the number of samples (10278) in class 3 will be larger than the number of samples in the majority class (class #2 -> 9650)
  n_samples_majority,


Starting Loop 72


C:\Users\rozhouse\Anaconda3\lib\site-packages\imblearn\utils\_validation.py:326: UserWarning: After over-sampling, the number of samples (10350) in class 3 will be larger than the number of samples in the majority class (class #2 -> 9635)
  n_samples_majority,


Starting Loop 73


...

Starting Loop 661


C:\Users\rozhouse\Anaconda3\lib\site-packages\imblearn\utils\_validation.py:326: UserWarning: After over-sampling, the number of samples (10152) in class 3 will be larger than the number of samples in the majority class (class #1 -> 5601)
  n_samples_majority,
C:\Users\rozhouse\Anaconda3\lib\site-packages\imblearn\utils\_validation.py:326: UserWarning: After over-sampling, the number of samples (5982) in class 7 will be larger than the number of samples in the majority class (class #1 -> 5601)
  n_samples_majority,


Starting Loop 662


C:\Users\rozhouse\Anaconda3\lib\site-packages\imblearn\utils\_validation.py:326: UserWarning: After over-sampling, the number of samples (10404) in class 3 will be larger than the number of samples in the majority class (class #1 -> 5559)
  n_samples_majority,
C:\Users\rozhouse\Anaconda3\lib\site-packages\imblearn\utils\_validation.py:326: UserWarning: After over-sampling, the number of samples (5760) in class 7 will be larger than the number of samples in the majority class (class #1 -> 5559)
  n_samples_majority,


Starting Loop 663
Starting Loop 664
Starting Loop 665
Starting Loop 666
Starting Loop 667
Starting Loop 668
Starting Loop 669
Starting Loop 670
Starting Loop 671


# Converting Results to Dataframe and Saving to Csv:

In [316]:
test_res_df_samp= pd.DataFrame(test_results, columns= ['av_val_loss', "av_accuracy", 'tomek', 'under_strat',
                            'over_strat', 'time_taken', 'av_epochs', 'cv_1_val_loss',
                                'cv_2_val_loss', 'cv_3_val_loss', 'cv_4_val_loss'])

In [317]:
tomek_map = {0: 'TL', 1: 'No TL'}
under_map = {0: '-15%', 1: '-25%', 2: '-40%', 3: '-60%', 4: '-80%', 5: 'Imbal'}
over_map = {0: '+20% S', 1: '+50% S', 2: '+100% S', 3: '+300% S', 4: '+500% S', 5: 'Imbal S', 6: 'SMOTE',
            7: '+20% BS', 8: '+50% BS', 9: '+100% BS', 10: '+300% BS', 11: '+500% BS', 12: 'Imbal BS', 13: 'BoSMOTE'}

In [318]:
test_res_df_samp['tomek'] = test_res_df_samp['tomek'].map(tomek_map)
test_res_df_samp['under_strat'] = test_res_df_samp['under_strat'].map(under_map)
test_res_df_samp['over_strat'] = test_res_df_samp['over_strat'].map(over_map)

In [4]:
test_res_df_samp.sort_values(by='av_val_loss')

,av_val_loss,tomek,under_strat,over_strat,time_taken,av_epochs,cv_1_val_loss,cv_2_val_loss,cv_3_val_loss,cv_4_val_loss
0,0.124310,TL,-15%,SMOTE,8896.437500,88.88,0.109355,0.182153,0.108922,0.096809
1,0.126080,TL,-15%,BoSMOTE,7236.890625,75.38,0.183616,0.080773,0.119248,0.120682
2,0.139486,No TL,-15%,SMOTE,7637.843750,82.13,0.162788,0.102508,0.158336,0.134314
3,0.143836,No TL,Imbal,SMOTE,4108.640625,86.63,0.140898,0.130114,0.180991,0.123340
4,0.144682,TL,-80%,+500% S,3300.812500,94.50,0.076449,0.109514,0.235394,0.157372
5,0.157577,TL,-60%,SMOTE,4012.296875,94.50,0.133853,0.204773,0.116076,0.175606
6,0.161531,TL,Imbal,SMOTE,4292.234375,100.00,0.251552,0.168233,0.150311,0.076026
7,0.165127,TL,Imbal,BoSMOTE,4008.265625,92.25,0.260163,0.188745,0.088149,0.123450
8,0.165936,TL,-80%,+300% S,2517.468750,97.88,0.237322,0.153238,0.105844,0.167340
9,0.166525,TL,-40%,BoSMOTE,4431.265625,69.75,0.198248,0.181117,0.129114,0.157621


In [321]:
test_res_df_samp.to_csv('Data/MLP_Gridsearch_Sampling_results.csv')